In [3]:
from google.colab import drive
ROOT = "/content/gdrive"
drive.mount(ROOT, force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
import sys
import os
folder = '/content/gdrive/My Drive/Colab Notebooks/my_project1/deep-learning/utils'
os.chdir(folder)

In [5]:
import torch.nn as nn
from helper import HelperModel
from dataset_info import DataSetInfo
from dataloader import DataLoader
from models.resnet import ResNet18, ResNet, BasicBlock
from models.resnet_tinyimagenet import ResNet18_GradCam
from transforms import Transform
from train import Train
from test import Test
from plot import Plot
from albumentations import *
from albumentations.pytorch import ToTensor
from gradcam import *
from tinymagenetdataset import ProcessTinyImageNet
from torchvisiondataset import TorchVisionDataSet
import matplotlib.pyplot as plt
from lr_finder import LRFinder
from torch.utils.data import random_split

  pip install torch-lr-finder -v --global-option="amp"


In [36]:
model_obj = ResNet18_GradCam(num_classes=200)
#import torchvision.models as models
#Load Resnet
# model_obj = models.resnet18()
# model_obj.avgpool = nn.AdaptiveAvgPool2d(1)
# model_obj.fc.out_features = 200

# create object for Model
device = HelperModel.get_device()

model = model_obj.to(device)

#create object for helper with object
helper_obj = HelperModel(model_obj)

#display summaru of model
helper_obj.display_model_summay(model, (3, 64, 64))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
            Conv2d-3           [-1, 64, 64, 64]          36,864
       BatchNorm2d-4           [-1, 64, 64, 64]             128
            Conv2d-5           [-1, 64, 64, 64]          36,864
       BatchNorm2d-6           [-1, 64, 64, 64]             128
        BasicBlock-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
           Conv2d-10           [-1, 64, 64, 64]          36,864
      BatchNorm2d-11           [-1, 64, 64, 64]             128
       BasicBlock-12           [-1, 64, 64, 64]               0
           Conv2d-13          [-1, 128, 32, 32]          73,728
      BatchNorm2d-14          [-1, 128,

In [5]:
url = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
HelperModel.download_extract_images(url)
#!unzip -q tiny-imagenet-200.zip

In [21]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from tqdm import notebook
import random
import pdb


class ProcessTinyImageNet():

    path = "tiny-imagenet-200/"
    #path ='/content/gdrive/My Drive/tiny-imagenet-200/'

    def __init__(self, classes_map):
        self.image = []
        self.target = []
        self.classes = classes_map
        self.update_train_val_data()

    @staticmethod
    def get_train_image_ids():
        """
            returns: ['n02321529',...]
        """
        path = ProcessTinyImageNet.path + "wnids.txt"
        train_image_ids = [line.strip() for line in open(path)]
        return train_image_ids

    @staticmethod
    def all_class_map():
        """
        eg:
        {'n02423022': 'label1'}
        """
        all_class_map = {}
        windis = ProcessTinyImageNet.get_train_image_ids()
        for line in open(ProcessTinyImageNet.path + "words.txt", 'r'):
            data = line.split('\t')[:2]
            n_id = data[0]
            class_name = data[1].strip()
            if n_id in windis:
              all_class_map[n_id] = class_name
        return all_class_map

    @staticmethod
    def get_image(image_path):
        img = Image.open(image_path)
        npimg = np.asarray(img)
        if len(npimg.shape) == 2:
            npimg = np.repeat(npimg[:, :, np.newaxis], 3, axis=2)
        return npimg

    def update_train_data(self):
        """
        returns: {'train/n02124075/images/n02124075_10.JPEG': 'Egyptian cat'}
        """
        image_ids = ProcessTinyImageNet.get_train_image_ids()
        class_map = self.classes
        train_image_path = ProcessTinyImageNet.path + "train/{image_id}/images/{image_name}.JPEG"
        for image_id in notebook.tqdm(image_ids, desc='Loading train folder...'):
            for key_id in range(500):
                image_name = image_id + "_" + str(key_id)
                train_path = train_image_path.format(image_id=image_id, image_name=image_name)
                #npimg = ProcessTinyImageNet.get_image(train_path)
                train_label = class_map[image_id]
                self.image.append(train_path)
                self.target.append(train_label)

    def update_val_data(self):
        """
        returns: {'val/images/val_10.JPEG': 'swimming trunks, bathing trunks'}
        """
        path = ProcessTinyImageNet.path + "val/val_annotations.txt"
        class_map = self.classes
        val_image_path = ProcessTinyImageNet.path + "val/images/{image_name}"
        for line in notebook.tqdm(open(path, 'r'), desc='Loading val folder....'):
            data = line.split('\t')[:2]
            image_name = data[0].strip()
            image_id = data[1].strip()
            image_path = val_image_path.format(image_name=image_name)
            #npimg = ProcessTinyImageNet.get_image(image_path)
            val_label = class_map[image_id]
            self.image.append(image_path)
            self.target.append(val_label)

    def update_train_val_data(self):
        self.update_train_data()
        self.update_val_data()

In [22]:
classes_map = ProcessTinyImageNet.all_class_map()
obj = ProcessTinyImageNet(classes_map)
#targets = torch.as_tensor(obj.target)
dataset = list(zip(obj.image, obj.target))
random.shuffle(dataset)
trainlen = int(len(dataset)*0.7)
classes = list(classes_map.values())
print(len(dataset), trainlen, len(classes))


110000 77000 200


In [15]:
import pickle
# train_set, test_set, classes = data_set_obj.get_tiny_imagenet_dataset(train_split = 70,train_transforms = train_transforms, test_transforms = test_transforms)
# mylist = [dataset, classes]
# with open('dataset.pkl', 'wb') as f:
#      pickle.dump(mylist, f)

with open('dataset.pkl', 'rb') as f:
    mylist = pickle.load(f)

dataset = mylist[0]
classes = mylist[1]

In [23]:

import torch
from skimage import io
import numpy as np
from torch.utils.data import Dataset
from sklearn import preprocessing
from PIL import Image

class TinyImagenetDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data, transform=None):
        """
        Args:
            data (string): zipped images and labels.
        """
        self.transform = transform
        self.images, self.labels = zip(*data)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = io.imread(self.images[idx], as_gray=False, pilmode="RGB")
        # image = Image.open(self.images)
        # npimg = np.asarray(image)
        # if (len(npimg.shape) == 2):
        #     npimg = np.repeat(npimg[:, :, np.newaxis], 3, axis=2)
        le = preprocessing.LabelEncoder()
        targets = le.fit_transform(self.labels)
        self.labels = torch.as_tensor(targets)
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx] #label

In [24]:
import albumentations as A
class Albumentation:
  """
  Helper class to create test and train transforms using Albumentations
  """
  def __init__(self, transforms=[]):
    self.transforms = A.Compose(transforms)

  def __call__(self, img):
    img = np.array(img)
    return self.transforms(image=img)['image']


### Apply Albumentation

In [25]:
patch_size = 32
mean = (0.48043839, 0.44820218, 0.39760034)
std = (0.27698959, 0.26908774, 0.28216029)
custom_transforms = [
       PadIfNeeded(min_height=70, min_width=70),
       RandomCrop(64, 64, p=1.0),
       HorizontalFlip(p=0.25),
       Rotate(limit=15, p=0.25),
       Normalize(mean=mean, std=std),
       Cutout(num_holes=4, max_h_size=16, max_w_size=16),
       ToTensor(),
    ]

train_transforms = Albumentation(custom_transforms)
test_transforms = Albumentation(
    [
     Normalize(mean=mean, std=std),
     ToTensor(),
     ])


In [26]:
# train_dataset = TorchVisionDataSet(train, transform=train_transforms)
# test_dataset = TorchVisionDataSet(test, transform=test_transforms)
train_dataset = TinyImagenetDataset(dataset[:trainlen], transform=train_transforms)
test_dataset = TinyImagenetDataset(dataset[trainlen:], transform=test_transforms)

### Get Train and Test data set

### Load the train and test data

In [27]:
data_loader_obj = DataLoader(train_dataset, test_dataset, batch_size=256)
train_loader = data_loader_obj.train_loader
test_loader = data_loader_obj.test_loader

CUDA Available? True


In [28]:
import matplotlib.pyplot as plt
dataiter = iter(train_loader)
images, labels = dataiter.next()
# show images
#plt.imshow(images)
# print labels
#print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [20]:
print(images.shape)

torch.Size([256, 3, 64, 64])


In [21]:
print(labels.shape)

torch.Size([256])


In [23]:
dataiter2 = iter(test_loader)
images1, labels1 = dataiter2.next()

In [24]:
print(images1.shape)
print(labels1.shape)

torch.Size([256, 3, 64, 64])
torch.Size([256])


### Display some training images to visualize the effect of cutout

In [44]:
optimizer = helper_obj.get_l2_regularizer(weight_decay=0.005, lr=1e-3, momentum=0.9) 
criterion = nn.CrossEntropyLoss() # As softmax added in model
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr = 0.04083366522928122, total_steps=None, epochs=50,
                                                steps_per_epoch=len(train_loader), pct_start=5/50, anneal_strategy='linear',
                                                cycle_momentum=False, base_momentum=0.9, max_momentum=0.95, div_factor=10, final_div_factor=10)

### **Run the range test to find the Max lr**

In [30]:
num_batches = np.floor(len(train_loader.dataset)/256)
total_epochs = 2
num_iter = total_epochs * num_batches
lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
lr_finder.range_test(train_loader, start_lr=1e-4, end_lr=1,num_iter=int(num_iter))
lr_finder.plot(plot_case="acc")
best_lr = lr_finder.get_best_lr(case="acc")

In [ ]:
lr_finder.plot(plot_case="loss")
best_lr = lr_finder.get_best_lr(case="loss")

### Reset the model to initial State

In [27]:
lr_finder.reset()

In [28]:
import torch
def save_model(model, optimizer, criterion, scheduler,  epoch, train_acc, train_loss, lr, test_acc, test_loss, file_name):
    """

    :param model: model which needs to be saved
    :param file_name: the file name of model eg: 'model_aug.pth'
    :return: None
    """
    checkpoint = {'state_dict': model.state_dict(),
                  'optimizer': optimizer.state_dict(),
                  'scheduler': scheduler.state_dict(),
                  'criterion':  criterion,
                  'epochs': epoch,
                  'train_acc': train_acc,
                  'train_loss': train_loss,
                  'lr':lr,
                  'test_acc': test_acc,
                  'test_loss': test_loss
                  }
    torch.save(checkpoint, file_name)

### Use OneCycleLR for scheduler and train the model

In [ ]:
max_lr = 0.04083366522928122  #best lr from lrfinder w.r.t max accuracy
EPOCHS = 50
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr = max_lr, total_steps=None, epochs=EPOCHS,
                                                steps_per_epoch=len(train_loader), pct_start=5/EPOCHS, anneal_strategy='linear',
                                                cycle_momentum=False, base_momentum=0.9, max_momentum=0.95, div_factor=10, final_div_factor=10)
train_obj = Train()
test_obj = Test()
train_acc = []
train_loss = []
lr_update = []
for epoch in range(1, EPOCHS+1):
    lr_update.append(optimizer.param_groups[0]['lr'])
    print("EPOCH:", epoch, "LR: ",lr_update[-1])
    train_obj.train(model, device, train_loader, optimizer, criterion, scheduler=scheduler)
    if epoch < EPOCHS:
        test_obj.test(model, device, test_loader, criterion, misclassfied_required=False, trueclassified_required=False, classes=classes, class_accuracy=90)
    else:
        # store only last epochsreult for misclassfied and trueclassified images
        test_obj.test(model, device, test_loader, criterion, misclassfied_required=True, trueclassified_required=True, classes=classes, class_accuracy=90)
    train_acc.append(train_obj.train_acc[-1])
    train_loss.append(train_obj.train_losses[-1])
    if epoch % 5 == 0:
      file_name = "tiny_imagenet_{0}.pth".format(epoch)
      save_model(model, optimizer, criterion, scheduler,  epoch, train_acc, train_loss, lr_update, test_obj.test_acc, test_obj.test_losses, file_name)

misclassified_images = test_obj.misclassified_images
correctcassified_images = test_obj.trueclassified_images
test_losses = test_obj.test_losses
test_acc = test_obj.test_acc
print("Maximum Training Accuracy = ", max(train_acc))
print("Maximum Testing Accuracy = ", max(test_acc))

  0%|          | 0/301 [00:00<?, ?it/s]

EPOCH: 1 LR:  0.0040833665229281215


Train Set: Train Loss=4.025679111480713 Batch_id=300 Accuracy=6.05: 100%|██████████| 301/301 [11:32<00:00,  2.30s/it]
  0%|          | 0/301 [00:00<?, ?it/s]


Test set: Average loss: 0.0167, Accuracy: 3111/33000 (9.43%)

EPOCH: 2 LR:  0.011438313272005436


Train Set: Train Loss=3.864513874053955 Batch_id=300 Accuracy=13.35: 100%|██████████| 301/301 [11:50<00:00,  2.36s/it]
  0%|          | 0/301 [00:00<?, ?it/s]

...

Train Set: Train Loss=2.2441844940185547 Batch_id=300 Accuracy=49.88: 100%|██████████| 301/301 [11:10<00:00,  2.23s/it]
  0%|          | 0/301 [00:00<?, ?it/s]


Test set: Average loss: 0.0099, Accuracy: 13220/33000 (40.06%)

EPOCH: 38 LR:  0.012083780387485036


Train Set: Train Loss=2.15421724319458 Batch_id=300 Accuracy=51.18: 100%|██████████| 301/301 [11:27<00:00,  2.28s/it]
  0%|          | 0/301 [00:00<?, ?it/s]


Test set: Average loss: 0.0093, Accuracy: 14328/33000 (43.42%)

EPOCH: 39 LR:  0.011185439752440848


Train Set: Train Loss=2.025883197784424 Batch_id=300 Accuracy=52.33: 100%|██████████| 301/301 [12:28<00:00,  2.49s/it]


In [32]:
def load_checkpoint(model, optimizer, scheduler, criterion,filename='checkpoint.pth'):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epochs']
        criterion = checkpoint['criterion']
        train_acc = checkpoint['train_acc']
        train_loss = checkpoint['train_loss']
        lr = checkpoint['lr']
        test_acc = checkpoint['test_acc']
        test_loss = checkpoint['test_loss']
        model = model.load_state_dict(checkpoint['state_dict'])
        optimizer = optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler = scheduler.load_state_dict(checkpoint['scheduler'])
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epochs']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))
    return model, optimizer, scheduler, criterion, start_epoch, train_acc, train_loss, lr, test_acc, test_loss

In [38]:
file_name = "tiny_imagenet_35.pth"
model, optimizer, scheduler, criterion, start_epoch, train_acc, train_loss, lr, test_acc, test_loss= load_checkpoint(model, optimizer, scheduler, criterion, filename=file_name)

=> loading checkpoint 'tiny_imagenet_35.pth'
=> loaded checkpoint 'tiny_imagenet_35.pth' (epoch 35)


None


In [45]:
max_lr = 0.04083366522928122  #best lr from lrfinder w.r.t max accuracy
EPOCHS = 50
start_epoch= start_epoch+1
train_obj = Train()
test_obj = Test()
train_acc = train_acc
train_loss = train_loss
lr_update = lr
for epoch in range(start_epoch, EPOCHS+1):
    lr_update.append(optimizer.param_groups[0]['lr'])
    print("EPOCH:", epoch, "LR: ",lr_update[-1])
    train_obj.train(model, device, train_loader, optimizer, criterion, scheduler=scheduler)
    if epoch < EPOCHS:
        test_obj.test(model, device, test_loader, criterion, misclassfied_required=False, trueclassified_required=False, classes=classes, class_accuracy=90)
    else:
        # store only last epochsreult for misclassfied and trueclassified images
        test_obj.test(model, device, test_loader, criterion, misclassfied_required=True, trueclassified_required=True, classes=classes, class_accuracy=90)
    train_acc.append(train_obj.train_acc[-1])
    train_loss.append(train_obj.train_losses[-1])
    if epoch % 5 == 0:
      file_name = "tiny_imagenet_{0}.pth".format(epoch)
      save_model(model, optimizer, criterion, file_name, epoch, train_acc, train_loss, lr_update, test_obj.test_acc, test_obj.test_losses)

misclassified_images = test_obj.misclassified_images
correctcassified_images = test_obj.trueclassified_images
test_losses = test_obj.test_losses
test_acc = test_obj.test_acc
print("Maximum Training Accuracy = ", max(train_acc))
print("Maximum Testing Accuracy = ", max(test_acc))

EPOCH: 37 LR:  0.001


AttributeError: ignored

### **Visualizing gradcam results for miss classified images in all layers**

In [ ]:
layers = [model.layer1, model.resnet_layer1, model.layer2, model.layer3, model.resnet_layer3]
Plot.plot_gradcam_images(model, layers, miscassified_images, classes)

### **Visualize the CYCLIC triangle plot using learning rates which used for each epochs**

In [ ]:
Plot.plot_cycle_lr(EPOCHS, lr_update)

### **Plot graphs**

In [ ]:
plt_obj = Plot(train_max_acc, train_min_loss, test_acc, test_losses)
# Plot all graph
plt_obj.display_all_plot()
plt_obj.plot_train_graph()
plt_obj.plot_validation_graph()
plt_obj.plot_graph(plot_case="Accuracy")

### **Plot misclassified images**

In [ ]:
Plot.plot_classified_images(classes, miscassified_images, image_name="gradcam_misclssified.png")